In [ ]:
import rosbag

def explore_rosbag_topics(bag_file, max_messages=5):
    """
    Explores all topics in the ROS bag and previews a limited number of messages.

    Args:
        bag_file (str): Path to the ROS bag file.
        max_messages (int): Maximum number of messages to preview per topic.
    """
    try:
        with rosbag.Bag(bag_file, 'r') as bag:
            print("Exploring topics in the bag file:")
            topic_counts = {}

            for topic, msg, t in bag.read_messages():
                if topic not in topic_counts:
                    topic_counts[topic] = 0

                if topic_counts[topic] < max_messages:
                    print(f"Topic: {topic}")
                    print(f"Message type: {type(msg)}")
                    print(f"Message content (preview): {msg}")
                    print("-" * 40)
                    topic_counts[topic] += 1

    except Exception as e:
        print(f"Error exploring bag file: {e}")

#explore_rosbag_topics("./automated_one_blower_full.bag", max_messages=20)
explore_rosbag_topics("./output_part326.bag", max_messages=20)

In [ ]:
import rosbag

def explore_specific_topic(bag_file, topic_name, max_messages=5):
    """
    Explores a specific topic in the ROS bag and previews a limited number of messages.

    Args:
        bag_file (str): Path to the ROS bag file.
        topic_name (str): The name of the topic to explore.
        max_messages (int): Maximum number of messages to preview.
    """
    try:
        with rosbag.Bag(bag_file, 'r') as bag:
            print(f"Exploring topic: {topic_name}")
            message_count = 0

            for topic, msg, t in bag.read_messages(topics=[topic_name]):
                if message_count < max_messages:
                    print(f"Message type: {type(msg)}")
                    print(f"Message content (preview): {msg}")
                    print(f"Timestamp: {t}")
                    print("-" * 40)
                    message_count += 1
                else:
                    break

            if message_count == 0:
                print(f"No messages found for topic '{topic_name}' in the bag file.")
    except Exception as e:
        print(f"Error exploring topic '{topic_name}': {e}")

# Example usage
explore_specific_topic("./automated_one_blower_full.bag", "/uav_status", max_messages=20)


In [ ]:
# Use this code!

import rosbag
import os
import rospy

def split_and_filter_bag(input_bag, output_prefix, topics=None, max_messages=1000, resume=True):
    """
    Split a bag file into smaller bags, optionally filtering by topic, with resume functionality.
    
    :param input_bag: Path to the input bag file.
    :param output_prefix: Prefix for the output bag files.
    :param topics: List of topics to include in the output (None = all topics).
    :param max_messages: Maximum number of messages per output file.
    :param resume: Whether to resume from the last processed message.
    """
    progress_file = f"{output_prefix}_progress.txt"
    last_timestamp = None
    part = 1
    count = 0

    # Resume from last progress
    if resume and os.path.exists(progress_file):
        with open(progress_file, "r") as f:
            last_timestamp = float(f.readline().strip())
            part = int(f.readline().strip())
            count = int(f.readline().strip())
        last_timestamp = rospy.Time.from_sec(last_timestamp)  # Convert float to rospy.Time

    bag = rosbag.Bag(input_bag)
    current_bag = None

    try:
        for topic, msg, t in bag.read_messages(topics=topics, start_time=last_timestamp):
            if count % max_messages == 0:
                # Close the current bag if open
                if current_bag:
                    current_bag.close()
                # Open a new bag file
                current_bag = rosbag.Bag(f"{output_prefix}_part{part}.bag", 'w')
                part += 1

            # Ensure `current_bag` is not None before writing
            if current_bag:
                current_bag.write(topic, msg, t)
            count += 1

            # Save progress every 100 messages
            if count % 100 == 0:
                with open(progress_file, "w") as f:
                    f.write(f"{t.to_sec()}\n")  # Record last processed timestamp
                    f.write(f"{part}\n")
                    f.write(f"{count}\n")
    finally:
        # Ensure the last open bag file is closed
        if current_bag:
            current_bag.close()
        bag.close()

# Give the name of the bag, the name of the output, the topics wanted and how many messages each sub bag should have
split_and_filter_bag(
    "./2025-01-09-14-25-10.bag",
    "output",
    topics=["/bus0/ft_sensor0/ft_sensor_readings/wrench", "/uav_status"],
    max_messages=100000
)


In [ ]:
# Use this code!



import os
import re
import rosbag
import numpy as np
import pandas as pd

def process_output_bags(output_prefix, wrench_topic, motor_topic, motor_ids, recovery_file="recovery_temp.csv"):
    combined_data = []  # To hold all rows of data
    processed_files = set()
    global_start_time = None  # Track the start time across all bags
    last_time_offset = 0  # Track the time offset from the last bag

    # Check for recovery file and load it if it exists
    if os.path.exists(recovery_file):
        print(f"Loading recovery data from {recovery_file}...")
        recovery_df = pd.read_csv(recovery_file)
        combined_data = recovery_df.values.tolist()
        # Extract already processed files from the DataFrame
        if "processed_file" in recovery_df.columns:
            processed_files = set(recovery_df["processed_file"].unique())
        # Extract the last time offset for continuity
        if "time" in recovery_df.columns:
            last_time_offset = recovery_df["time"].max()
        print(f"Recovered {len(processed_files)} files from previous run.")

    # List and sort the output bag files
    bag_files = sorted(
        [f for f in os.listdir('.') if re.match(f"{output_prefix}_part\\d+\\.bag", f)],
        key=lambda x: int(re.search(r"\d+", x).group())  # Correct regex for extracting numbers
    )

    last_known_setpoints = {motor_id: float('nan') for motor_id in motor_ids}  # Initialize with NaN

    for bag_file in bag_files:
        # Skip files already processed
        if bag_file in processed_files:
            print(f"Skipping {bag_file}, already processed.")
            continue

        try:
            print(f"Processing {bag_file}...")
            with rosbag.Bag(bag_file, 'r') as bag:
                bag_start_time = None  # Start time for the current bag

                # Iterate through messages in both topics
                for topic, msg, t in bag.read_messages(topics=[wrench_topic, motor_topic]):
                    if topic == wrench_topic:
                        # Set global start time if not already set
                        if global_start_time is None:
                            global_start_time = t.to_sec()
                        if bag_start_time is None:
                            bag_start_time = t.to_sec()
                        
                        # Calculate continuous time
                        time_relative = last_time_offset + (t.to_sec() - bag_start_time)
                        row = [
                            time_relative,    # Continuous relative time
                            msg.wrench.force.z,  # Only extract force_z
                        ]
                        # Add the last known setpoints for motor IDs
                        row += [last_known_setpoints[motor_id] for motor_id in motor_ids]
                        row.append(bag_file)  # Add file name for recovery
                        combined_data.append(row)

                    elif topic == motor_topic:
                        # Extract motor setpoints for the specified motor IDs
                        setpoints = {motor.id: motor.setpoint for motor in msg.motors}
                        # Update last known setpoints
                        for motor_id in motor_ids:
                            if motor_id in setpoints:
                                last_known_setpoints[motor_id] = setpoints[motor_id]

                # Update last time offset after processing the bag
                if bag_start_time is not None:
                    last_time_offset += (t.to_sec() - bag_start_time)

            # Save progress to the recovery file after processing each bag
            recovery_df = pd.DataFrame(
                combined_data, 
                columns=['time', 'force_z'] + [f'motor_{motor_id}_setpoint' for motor_id in motor_ids] + ['processed_file']
            )
            recovery_df.to_csv(recovery_file, index=False)
            print(f"Progress saved to {recovery_file}.")

        except Exception as e:
            print(f"Error processing {bag_file}: {e}")

    # Convert the combined data into a NumPy array
    combined_array = np.array([row[:-1] for row in combined_data])  # Exclude 'processed_file' column for the final array
    print("Combined data extraction complete.")
    print(f"Array shape: {combined_array.shape}")
    return combined_array

# Example usage
output_prefix = "output"
wrench_topic = "/bus0/ft_sensor0/ft_sensor_readings/wrench"
motor_topic = "/uav_status"  # Replace with your actual motor topic
motor_ids = [0, 1, 6, 7]  # Motor IDs to extract
recovery_file = "recovery_temp.csv"

data_array = process_output_bags(output_prefix, wrench_topic, motor_topic, motor_ids, recovery_file)

# Define column names for the DataFrame
columns = ['time', 'force_z'] + [f'motor_{motor_id}_setpoint' for motor_id in motor_ids]
columns = ['time', 'force_z', 'rpm_left', 'rpm_measure', 'angle_left', 'angle_measure']
df = pd.DataFrame(data_array, columns=columns)

# Save the final output to a CSV
output_csv = "combined_output.csv"
df.to_csv(output_csv, index=False)
print(f"Final data saved to {output_csv}.")


In [ ]:
print(df.head())

In [ ]:
# Save the DataFrame as a CSV file
#csv_path = "./z_data.csv"  # File will be saved in the same folder as the notebook
#df.to_csv(csv_path)

In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv("combined_output.csv")
data_array = df.to_numpy()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_rosbag_data_direct(extracted_data):
    """
    Plots forces, torques, RPMs, and angles directly from the extracted data using Plotly.

    Args:
        extracted_data (np.ndarray): 2D array with columns:
                                     [time, force_x, force_y, force_z,
                                      torque_x, torque_y, torque_z,
                                      id_0 (rpm_0), id_1 (rpm_1),
                                      id_6 (angle_100), id_7 (angle_101)].
    """
    # Extract data columns
    #time = extracted_data[:, 0][0:-1:100]
    ##force_x = extracted_data[:, 1]
    ##force_y = extracted_data[:, 2]
    #force_z = extracted_data[:, 1][0:-1:100]
    ##torque_x = extracted_data[:, 4]
    ##torque_y = extracted_data[:, 5]
    ##torque_z = extracted_data[:, 6]
    #rpm_0 = extracted_data[:, 2][0:-1:100]
    #rpm_1 = extracted_data[:, 3][0:-1:100]
    #angle_100 = extracted_data[:, 4][0:-1:100]
    #angle_101 = extracted_data[:, 5][0:-1:100]

    time = extracted_data[:, 0][0:-1:100]
    #force_x = extracted_data[:, 1]
    #force_y = extracted_data[:, 2]
    force_z = extracted_data[:, 1][0:-1:100]
    #torque_x = extracted_data[:, 4]
    #torque_y = extracted_data[:, 5]
    #torque_z = extracted_data[:, 6]
    rpm_0 = extracted_data[:, 2][0:-1:100]
    rpm_1 = extracted_data[:, 3][0:-1:100]
    angle_100 = extracted_data[:, 4][0:-1:100]
    angle_101 = extracted_data[:, 5][0:-1:100]

    components = [
        #("Force X", force_x, "Force (N)", "blue"),
        #("Torque X", torque_x, "Torque (Nm)", "orange"),
        #("Force Y", force_y, "Force (N)", "green"),
        #("Torque Y", torque_y, "Torque (Nm)", "purple"),
        ("Force Z", force_z, "Force (N)", "red"),
        #("Torque Z", torque_z, "Torque (Nm)", "brown"),
    ]

    for title, data, y_label, color in components:
        # Create subplots with shared x-axis
        fig = make_subplots(
            rows=2, cols=1, shared_xaxes=True,
            subplot_titles=(title, "RPM and Angle Data"),
            specs=[[{"secondary_y": False}], [{"secondary_y": True}]]
        )

        # Add main component data to the first subplot
        fig.add_trace(
            go.Scatter(x=time, y=data, mode='lines', name=title, line=dict(color=color)),
            row=1, col=1
        )

        # Add RPM data to the second subplot (left y-axis)
        fig.add_trace(
            go.Scatter(x=time, y=rpm_0, mode='lines',
                       name="RPM Left (id_0)", line=dict(color='purple', dash='dash')),
            row=2, col=1, secondary_y=False
        )
        fig.add_trace(
            go.Scatter(x=time, y=rpm_1, mode='lines',
                       name="RPM Measure (id_1)", line=dict(color='purple', dash='dot')),
            row=2, col=1, secondary_y=False
        )

        # Add angle data to the second subplot (right y-axis)
        fig.add_trace(
            go.Scatter(x=time, y=angle_100, mode='lines',
                       name="Angle Left (id_6)", line=dict(color='orange', dash='solid')),
            row=2, col=1, secondary_y=True
        )
        fig.add_trace(
            go.Scatter(x=time, y=angle_101, mode='lines',
                       name="Angle Measure (id_7)", line=dict(color='green', dash='solid')),
            row=2, col=1, secondary_y=True
        )

        # Update layout
        fig.update_layout(
            title=f'{title} over Time with RPM and Angle Data',
            xaxis_title='Time (s)',
            yaxis_title=y_label,
            template='plotly_white'
        )

        # Update secondary y-axis for the second subplot
        fig.update_yaxes(title_text="Angles", secondary_y=True, row=2, col=1)
        fig.update_yaxes(title_text="RPM", secondary_y=False, row=2, col=1)

        fig.show()

plot_rosbag_data_direct(data_array)

In [ ]:
import pandas as pd
import numpy as np

def calculate_means_between_points(filtered_wrench, time_data, time_pairs, component):
    """
    Calculates the mean of filtered wrench data between defined pairs of time points.

    Args:
        filtered_wrench (dict): Dictionary containing filtered wrench data.
        time_data (list or np.array): Time series corresponding to the data.
        time_pairs (list of tuples): Pairs of time points [(t1_start, t1_end), (t2_start, t2_end), ...].
        component (str): The wrench component to analyze (e.g., 'force_x', 'torque_y').

    Returns:
        list: Mean values for each time pair.
    """
    means = []
    for start, end in time_pairs:
        # Mask to select data between start and end times
        mask = (np.array(time_data) >= start) & (np.array(time_data) <= end)
        selected_data = np.array(filtered_wrench[component])[mask]
        
        # Calculate mean and store it
        mean_value = np.mean(selected_data)
        means.append(mean_value)
        print(f"Mean of {component} between {start}s and {end}s: {mean_value:.4f}")
    return means

# Define 3 pairs of time points
time_pairs = [
    (92.7, 96.7),  # Pair 1: Influence
    (98.7, 101.5),  # Pair 2: Baseline
    (103.7, 105.5)   # Pair 3: Bias
]

# Select a wrench component to analyze, e.g., 'force_z'
component_to_analyze = "force_z"

# Extract time data
time_data = np.array(df["time"])

# Calculate means for the specified time pairs
means = calculate_means_between_points(df, time_data, time_pairs, component_to_analyze)

# Assign names to the means
influence = means[0]
baseline = means[1]
bias = means[2]

# Subtract bias from the baseline and influence
baseline_corrected = baseline - bias
influence_corrected = influence - bias

# Calculate absolute and relative influences
absolute_influence = influence_corrected - baseline_corrected
relative_influence = (absolute_influence / baseline_corrected) * 100 if baseline_corrected != 0 else np.nan

# Print the results
print(f"Bias: {bias:.4f}")
print(f"Baseline (corrected): {baseline_corrected:.4f}")
print(f"Absolute Influence: {absolute_influence:.4f}")
print(f"Relative Influence: {relative_influence:.2f}%")

In [ ]:
from scipy.fft import fft, fftfreq

def perform_fourier_transform_interactive_new(extracted_data, column_idx, start_time, end_time, title="Fourier Transform Analysis"):
    """
    Performs a Fourier Transform on the data between specified start and end times and plots interactively using Plotly.

    Args:
        extracted_data (np.ndarray): The 2D array containing time and data columns.
        column_idx (int): The index of the column in `extracted_data` to analyze.
        start_time (float): The starting time for the analysis.
        end_time (float): The ending time for the analysis.
        title (str): Title for the plot.
    """
    # Extract time and data columns
    time = extracted_data[:, 0]
    data = extracted_data[:, column_idx]

    # Select the data within the specified time range
    mask = (time >= start_time) & (time <= end_time)
    selected_time = time[mask]
    selected_data = data[mask]
    
    # Check if there are enough data points
    if len(selected_time) < 2:
        print("Not enough data points in the selected range.")
        return
    
    # Calculate the sampling frequency
    dt = np.mean(np.diff(selected_time))  # Average time step
    fs = 1 / dt  # Sampling frequency
    
    print(f"Sampling Frequency: {fs} Hz")
    
    # Perform Fourier Transform
    N = len(selected_data)
    yf = fft(selected_data)
    xf = fftfreq(N, dt)[:N // 2]  # Frequency range
    
    # Create an interactive plot
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=xf, y=2.0 / N * np.abs(yf[:N // 2]), mode='lines', name='Amplitude')
    )
    fig.update_layout(
        title=title,
        xaxis_title="Frequency (Hz)",
        yaxis_title="Amplitude",
        template="plotly_white",
        hovermode="x"
    )
    fig.show()

# Example Usage
# Perform Fourier Transform on Force Z (column index 3)
start_time = 1711.5  # First big bag, 7100 90 deg
end_time = 1712.6   # Replace with your desired end time

start_time = 428  # Second big bag, maximal negative influence 5800
end_time = 432   # Replace with your desired end time

start_time = 4338  # Second big bag, close to max pos influence, max uncertainty 5800
end_time = 4343   # Replace with your desired end time


start_time = 4345  # Second big bag, close to max pos influence, max uncertainty 5800
end_time = 4348   # Replace with your desired end time
perform_fourier_transform_interactive_new(
    extracted_data=data_array, 
    column_idx=1,  # Column index for Force Z
    start_time=start_time, 
    end_time=end_time, 
    title="Fourier Transform at Maximum Standard Deviation"
)


In [ ]:
def detect_changes(data, threshold=1e-6):
    """
    Detect changes in a data array and return the times and change values.

    Args:
        data (np.ndarray): 1D array of data (e.g., RPM or angle values).
        threshold (float): Minimum change threshold to consider.

    Returns:
        list: Indices at which changes occurred.
        list: Tuples of (old_value, new_value) for each change.
    """
    changes = []
    indices = []
    for i in range(1, len(data)):
        if (
            (np.isnan(data[i - 1]) and not np.isnan(data[i])) or  # NaN to valid number
            (not np.isnan(data[i - 1]) and np.isnan(data[i])) or  # Valid number to NaN
            (not np.isnan(data[i - 1]) and not np.isnan(data[i]) and abs(data[i] - data[i - 1]) > threshold)  # Significant value change
        ):
            indices.append(i)
            changes.append((data[i - 1], data[i]))
    return indices, changes

# Example usage of detect_changes function
time_data = data_array[:, 0]  # Time is in column 0 of the array
rpm_measure = data_array[:, 3]  # `rpm_measure` is column index 8
rpm_left = data_array[:, 2]  # `rpm_left` is column index 7

# Detect changes in `rpm_measure`
rpm_measure_change_indices, rpm_measure_changes = detect_changes(rpm_measure, threshold=1e-2)

# Detect changes in `rpm_left`
rpm_left_change_indices, rpm_left_changes = detect_changes(rpm_left, threshold=1e-2)

# Convert indices to corresponding time values for readability
rpm_measure_change_times = [time_data[idx] for idx in rpm_measure_change_indices]
rpm_left_change_times = [time_data[idx] for idx in rpm_left_change_indices]

# Print detected changes for `rpm_measure`
print("Detected Changes in RPM Measure:")
for time, change in zip(rpm_measure_change_times, rpm_measure_changes):
    print(f"At {time:.6f}s: {change[0]:.2f} -> {change[1]:.2f} of Measure")

# Print detected changes for `rpm_left`
print("\nDetected Changes in RPM Left:")
for time, change in zip(rpm_left_change_times, rpm_left_changes):
    print(f"At {time:.6f}s: {change[0]:.2f} -> {change[1]:.2f} of Left")

In [ ]:
def generate_time_groups_with_nan_handling(time_data, rpm_measure, rpm_left, threshold=1e-6, start_time=100):
    """
    Automatically generates groups of 3 pairs of time points based on changes
    in rpm_measure and rpm_left, while handling NaN transitions.

    Args:
        time_data (np.ndarray): Time series corresponding to the data.
        rpm_measure (np.ndarray): RPM measure data series.
        rpm_left (np.ndarray): RPM left data series.
        threshold (float): Minimum change threshold for detecting changes.
        start_time (float): The starting time for group generation (optional).

    Returns:
        list of list of tuples: Groups of 3 pairs of time points.
    """
    def find_next_transition(times, changes, start_time, condition):
        """Find the next transition time based on the condition."""
        for idx, t in enumerate(times):
            if t > start_time and condition(changes[idx]):
                return t
        return None

    time_groups = []
    wait_time = 2  # Wait 2 seconds as described

    # Detect changes in rpm_measure and rpm_left
    rpm_measure_indices, rpm_measure_changes = detect_changes(rpm_measure, threshold)
    rpm_left_indices, rpm_left_changes = detect_changes(rpm_left, threshold)

    # Convert indices to times
    rpm_measure_times = [time_data[idx] for idx in rpm_measure_indices]
    rpm_left_times = [time_data[idx] for idx in rpm_left_indices]

    # Debug: Print detected transitions
    print("Detected rpm_measure transitions:", list(zip(rpm_measure_times, rpm_measure_changes)))
    print("Detected rpm_left transitions:", list(zip(rpm_left_times, rpm_left_changes)))

    # Set the initial time for group generation
    current_time = start_time if start_time is not None else time_data[0]

    i = 0
    while i < len(rpm_measure_times) - 1:
        group = []

        # Start of the group: Detect when rpm_measure goes up (including NaN to value)
        if rpm_measure_times[i] < current_time:
            i += 1
            continue

        if (
            np.isnan(rpm_measure_changes[i][0]) or  # Transition from NaN to a value
            rpm_measure_changes[i][0] < rpm_measure_changes[i][1]  # Value increases
        ):
            # Pair 1: Wait 2 seconds, then find when rpm_left goes down (or NaN to value)
            t1_start = rpm_measure_times[i] + wait_time
            t1_end = find_next_transition(
                rpm_left_times,
                rpm_left_changes,
                t1_start,
                lambda change: np.isnan(change[0]) or change[0] > change[1]
            )

            if t1_end and 3.9 <= (t1_end - t1_start) <= 4.1:  # Validate Pair 1 duration
                group.append((t1_start, t1_end))

            # Pair 2: Wait 2 seconds after t1_end, then find when rpm_measure goes down (or value to NaN)
            if t1_end:
                t2_start = t1_end + wait_time
                t2_end = find_next_transition(
                    rpm_measure_times,
                    rpm_measure_changes,
                    t2_start,
                    lambda change: change[0] > change[1] or np.isnan(change[1])
                )

                if t2_end and 2.9 <= (t2_end - t2_start) <= 3.1:  # Validate Pair 2 duration
                    group.append((t2_start, t2_end))

            # Pair 3: Wait 2 seconds after t2_end, then find when rpm_measure goes up again (or NaN to value)
            if t2_end:
                t3_start = t2_end + wait_time
                t3_end = find_next_transition(
                    rpm_measure_times,
                    rpm_measure_changes,
                    t3_start,
                    lambda change: np.isnan(change[0]) or change[0] < change[1]
                )

                if t3_end and 2.9 <= (t3_end - t3_start) <= 3.1:  # Validate Pair 3 duration
                    group.append((t3_start, t3_end))

            # Add the group if all 3 pairs are valid
            if len(group) == 3:
                time_groups.append(group)
            else:
                print(f"Skipping incomplete or invalid group starting at index {i}: {group}")

        i += 1

    print(f"Total groups generated: {len(time_groups)}")
    return time_groups

# Example Usage
# Extract necessary columns from data_array
time_data = data_array[:, 0]  # Time is in the first column
rpm_measure = data_array[:, 3]  # rpm_measure is column index 8
rpm_left = data_array[:, 2]  # rpm_left is column index 7

# Generate time groups for multiple groups
time_groups = generate_time_groups_with_nan_handling(time_data, rpm_measure, rpm_left, threshold=1e-2, start_time=100)

# Output the generated time groups
print("Generated Time Groups for Multiple Groups:")
for group_idx, group in enumerate(time_groups, start=1):
    print(f"Group {group_idx}: {group}")



In [8]:
def calculate_means_for_groups_with_labels(data, time_data, groups, column_idx, angle_data, rpm_data):
    """
    Calculates the mean of data for each group of 3 pairs of time points, 
    labeling each group with constant angles and maximum speeds.

    Args:
        data (np.ndarray): Data array to analyze.
        time_data (np.ndarray): Time series corresponding to the data.
        groups (list of list of tuples): Groups of 3 pairs of time points.
                                          Each group is a list of tuples [(t1_start, t1_end), ...].
        column_idx (int): Column index in the data array to analyze.
        angle_data (np.ndarray): Array of angle data (e.g., `id_6` and `id_7` for angles).
        rpm_data (np.ndarray): Array of RPM data (e.g., `id_0` and `id_1` for speeds).

    Returns:
        list: Results for each group as a dictionary.
    """
    results = []

    for group_idx, group in enumerate(groups):
        if len(group) != 3:
            print(f"Skipping group {group_idx + 1}: Not exactly 3 pairs.")
            continue

        # Use the angles from the first pair of the group
        first_pair_mask = (time_data >= group[0][0]) & (time_data <= group[0][1])
        angle_id_6 = angle_data[first_pair_mask, 0][0]  # Take first value of id_6
        angle_id_7 = angle_data[first_pair_mask, 1][0]  # Take first value of id_7

        # Calculate the max RPMs for the entire group
        group_mask = np.any([(time_data > start) & (time_data < end) for start, end in group], axis=0)
        max_rpm_id_0 = np.max(rpm_data[group_mask, 0])  # Max RPM (id_0)
        max_rpm_id_1 = np.max(rpm_data[group_mask, 1])  # Max RPM (id_1)

        means = []
        for start, end in group:
            # Mask to select data between start and end times
            mask = (time_data >= start) & (time_data <= end)
            selected_data = data[mask, column_idx]

            # Calculate mean and handle empty selection gracefully
            mean_value = np.mean(selected_data) if len(selected_data) > 0 else np.nan
            means.append(mean_value)

        if len(means) == 3:
            influence, baseline, bias = means
            baseline_corrected = baseline - bias
            influence_corrected = influence - bias
            absolute_influence = influence_corrected - baseline_corrected
            relative_influence = (absolute_influence / baseline_corrected) * 100 if baseline_corrected != 0 else np.nan

            results.append({
                "group": group_idx + 1,
                "angle_id_6": angle_id_6,
                "angle_id_7": angle_id_7,
                "max_rpm_id_0": max_rpm_id_0,
                "max_rpm_id_1": max_rpm_id_1,
                "influence": influence,
                "baseline": baseline,
                "bias": bias,
                "baseline_corrected": baseline_corrected,
                "influence_corrected": influence_corrected,
                "absolute_influence": absolute_influence,
                "relative_influence": relative_influence,
            })

    return results


# Example Usage
# Assuming `data_array`, `time_data`, `angle_data`, `rpm_data`, and `groups` are defined
column_idx_to_analyze = 1
rpm_data = data_array[:, 2:4]
angle_data = data_array[:, 4:]
results = calculate_means_for_groups_with_labels(
    data_array, time_data, time_groups, column_idx_to_analyze, angle_data, rpm_data
)

# Convert results to a DataFrame
results_df = pd.DataFrame(results)


In [ ]:
#print(results_df.head())
print(results_df.to_string())

In [ ]:
# Save the final output to a CSV
results_csv = "results.csv"
results_df.to_csv(results_csv, index=False)
print(f"Final data saved to {results_csv}.")

In [1]:
import pandas as pd
import numpy as np
results_df = pd.read_csv("results.csv")

In [ ]:
# Group by `max_rpm_id_1` and describe the `baseline` column
baseline_description = results_df.groupby("max_rpm_id_1")["baseline_corrected"].describe()
print(baseline_description.to_string())
#

In [ ]:
#Main

# Convert results to a DataFrame
#results_df = pd.DataFrame(results)
results_df = pd.read_csv("results.csv")

import numpy as np
import plotly.graph_objects as go

# Group by speeds (max_rpm_id_0 and max_rpm_id_1)
speed_groups = results_df.groupby(["max_rpm_id_0", "max_rpm_id_1"])
# Convert angles from radians to degrees and round them
results_df['angle_id_6'] = np.degrees(results_df['angle_id_6']).round(1)  # Rounded to 1 decimal place
results_df['angle_id_7'] = np.degrees(results_df['angle_id_7']).round(1)
# Iterate through each speed group and create a separate plot
for i, ((rpm_0, rpm_1), group) in enumerate(speed_groups):
    # Create pivot table
    group = group.groupby(["angle_id_6", "angle_id_7"], as_index=False).agg({"relative_influence": "mean"})

    # Now perform the pivot operation
    pivot = group.pivot(index="angle_id_6", columns="angle_id_7", values="relative_influence")

    
    ## Debugging the pivot table
    #print(pivot.to_string())
    #print(pivot.index)  # Check the row indices
    #print(pivot.columns)  # Check the column indices
    #print(pivot.values)  # Check the data values


    # Extract x, y, and z for the surface
    x = pivot.columns.values  # Angle ID 7 (columns of the pivot)
    y = pivot.index.values    # Angle ID 6 (index of the pivot)
    z = pivot.values          # Relative influence (pivot values)
    #print("x shape:", x.shape)
    #print("y shape:", y.shape)
    #print("z shape:", z.shape)

    # Create a mask for NaN values
    nan_mask = np.isnan(pivot.values)

    # Replace NaN with zero for plotting
    filled_data = np.nan_to_num(pivot.values)

    z = filled_data
    # Ensure x, y are 2D for the surface
    x, y = np.meshgrid(x, y)
    # Ensure missing values are represented as NaN
    #z = np.where(np.isnan(pivot.values), None, pivot.values)


    # Identify NaN locations for marking
    nan_y, nan_x = np.where(nan_mask)  # Indices of NaN values in the array
    highlight_x = x[nan_y, nan_x]      # X-coordinates of NaNs
    highlight_y = y[nan_y, nan_x]      # Y-coordinates of NaNs
    highlight_z = np.zeros_like(highlight_x)  # Set Z to 0 for markers

    # Create a new figure
    fig = go.Figure()

    # Add surface for this speed group
    fig.add_trace(go.Surface(
        z=z,
        x=x,
        y=y,
        colorscale="Viridis",  # Use a color scale for this surface
        showscale=True,  # Show the color scale for this plot
        name=f"RPM Left: {rpm_0}, RPM Measure: {rpm_1}"  # Title for this surface
    ))

    # Add markers for NaN values
    fig.add_trace(go.Scatter3d(
        x=highlight_x,
        y=highlight_y,
        z=highlight_z,
        mode='markers',
        marker=dict(
            size=5,
            color='red',  # Color to mark NaN points
            symbol='circle'
        ),
        name='NaN Points'
    ))

    # Update layout
    fig.update_layout(
        title=f"3D Surface Plot with NaN Markers for RPM Left: {rpm_0}, RPM Measure: {rpm_1}",
        scene=dict(
            xaxis_title="Angle Measure",
            yaxis_title="Angle Left",
            zaxis_title="Relative Influence"
        ),
        template="plotly_white"
    )


    # Show the plot
    fig.show()


In [ ]:
#Scatter


# Convert results to a DataFrame
#results_df = pd.DataFrame(results)
results_df = pd.read_csv("results.csv")

import numpy as np
import plotly.graph_objects as go

# Group by speeds (max_rpm_id_0 and max_rpm_id_1)
speed_groups = results_df.groupby(["max_rpm_id_0", "max_rpm_id_1"])

# Convert angles from radians to degrees and round them
results_df['angle_id_6'] = np.degrees(results_df['angle_id_6']).round(1)  # Rounded to 1 decimal place
results_df['angle_id_7'] = np.degrees(results_df['angle_id_7']).round(1)

# Iterate through each speed group and create a separate plot
for i, ((rpm_0, rpm_1), group) in enumerate(speed_groups):
    # Aggregate data within the group
    group = group.groupby(["angle_id_6", "angle_id_7"], as_index=False).agg({"relative_influence": "mean"})

    # Create pivot table
    pivot = group.pivot(index="angle_id_6", columns="angle_id_7", values="relative_influence")

    # Convert pivot table to scatter data
    x, y, z = [], [], []

    # Iterate over the pivot table and extract non-NaN values for plotting
    for row_idx, row in enumerate(pivot.index):
        for col_idx, col in enumerate(pivot.columns):
            value = pivot.iat[row_idx, col_idx]
            if not np.isnan(value):  # Exclude NaN values
                x.append(col)  # Angle ID 7
                y.append(row)  # Angle ID 6
                z.append(value)  # Relative influence

    # Create scatter plot
    fig = go.Figure()

    # Add scatter points for this speed group
    fig.add_trace(go.Scatter3d(
        x=x,
        y=y,
        z=z,
        mode='markers',
        marker=dict(
            size=5,
            color=z,  # Color based on the relative influence
            colorscale='Viridis',  # Use a color scale
            showscale=True  # Show the color scale
        ),
        name=f"RPM Left: {rpm_0}, RPM Measure: {rpm_1}"  # Title for this group
    ))

    # Update layout
    fig.update_layout(
        title=f"3D Scatter Plot for RPM Left: {rpm_0}, RPM Measure: {rpm_1}",
        scene=dict(
            xaxis_title="Angle Measure",
            yaxis_title="Angle Left",
            zaxis_title="Relative Influence"
        ),
        template="plotly_white"
    )

    # Show the plot
    fig.show()



In [ ]:
#Interpolated

# Convert results to a DataFrame
#results_df = pd.DataFrame(results)
results_df = pd.read_csv("results.csv")

import numpy as np
import plotly.graph_objects as go

# Group by speeds (max_rpm_id_0 and max_rpm_id_1)
speed_groups = results_df.groupby(["max_rpm_id_0", "max_rpm_id_1"])

# Convert angles from radians to degrees and round them
results_df['angle_id_6'] = np.degrees(results_df['angle_id_6']).round(1)  # Rounded to 1 decimal place
results_df['angle_id_7'] = np.degrees(results_df['angle_id_7']).round(1)

# Iterate through each speed group and create a separate plot
for i, ((rpm_0, rpm_1), group) in enumerate(speed_groups):
    # Aggregate data within the group
    group = group.groupby(["angle_id_6", "angle_id_7"], as_index=False).agg({"relative_influence": "mean"})

    # Create pivot table
    pivot = group.pivot(index="angle_id_6", columns="angle_id_7", values="relative_influence")

    # Interpolate missing values
    pivot_interpolated = pivot.interpolate(method='linear', axis=0).interpolate(method='linear', axis=1)

    # Create a mask for original NaN values before interpolation
    nan_mask = pivot.isna()

    # Extract x, y, and z for the surface
    x = pivot_interpolated.columns.values  # Angle ID 7 (columns of the pivot)
    y = pivot_interpolated.index.values    # Angle ID 6 (index of the pivot)
    z = pivot_interpolated.values          # Interpolated relative influence (pivot values)

    # Ensure x, y are 2D for the surface
    x, y = np.meshgrid(x, y)

    # Identify NaN locations for marking
    nan_y, nan_x = np.where(nan_mask.values)  # Indices of original NaN values
    highlight_x = x[nan_y, nan_x]             # X-coordinates of NaNs
    highlight_y = y[nan_y, nan_x]             # Y-coordinates of NaNs
    highlight_z = np.zeros_like(highlight_x)  # Set Z to 0 for markers

    # Create a new figure
    fig = go.Figure()

    # Add surface for this speed group
    fig.add_trace(go.Surface(
        z=z,
        x=x,
        y=y,
        colorscale="Viridis",  # Use a color scale for this surface
        showscale=True,  # Show the color scale for this plot
        name=f"RPM Left: {rpm_0}, RPM Measure: {rpm_1}"  # Title for this surface
    ))

    ## Add markers for original NaN values
    #fig.add_trace(go.Scatter3d(
    #    x=highlight_x,
    #    y=highlight_y,
    #    z=highlight_z,
    #    mode='markers',
    #    marker=dict(
    #        size=5,
    #        color='red',  # Color to mark original NaN points
    #        symbol='circle'
    #    ),
    #    name='Original NaN Points'
    #))

    # Update layout
    fig.update_layout(
        title=f"Interpolated 3D Surface Plot for RPM Left: {rpm_0}, RPM Measure: {rpm_1}",
        scene=dict(
            xaxis_title="Angle Measure",
            yaxis_title="Angle Left",
            zaxis_title="Relative Influence"
        ),
        template="plotly_white"
    )

    # Show the plot
    fig.show()


In [ ]:
# Convert results to a DataFrame
#results_df = pd.DataFrame(results)
results_df = pd.read_csv("results.csv")

import numpy as np
import plotly.graph_objects as go

# Group by speeds (max_rpm_id_0 and max_rpm_id_1)
speed_groups = results_df.groupby(["max_rpm_id_0", "max_rpm_id_1"])

# Convert angles from radians to degrees and round them
results_df['angle_id_6'] = np.degrees(results_df['angle_id_6']).round(1)  # Rounded to 1 decimal place
results_df['angle_id_7'] = np.degrees(results_df['angle_id_7']).round(1)

# Iterate through each speed group and create a separate plot
for i, ((rpm_0, rpm_1), group) in enumerate(speed_groups):
    # Aggregate data within the group
    group = group.groupby(["angle_id_6", "angle_id_7"], as_index=False).agg({"relative_influence": "mean"})

    # Create pivot table
    pivot = group.pivot(index="angle_id_6", columns="angle_id_7", values="relative_influence")

    # Interpolate missing values
    pivot_interpolated = pivot.interpolate(method='linear', axis=0).interpolate(method='linear', axis=1)

    # Create a mask for original NaN values before interpolation
    nan_mask = pivot.isna()

    # Extract x, y, and z for the surface
    x = pivot_interpolated.columns.values  # Angle ID 7 (columns of the pivot)
    y = pivot_interpolated.index.values    # Angle ID 6 (index of the pivot)
    z = pivot_interpolated.values          # Interpolated relative influence (pivot values)

    # Ensure x, y are 2D for the surface
    x, y = np.meshgrid(x, y)

    # Identify original points
    original_x = group["angle_id_7"]
    original_y = group["angle_id_6"]
    original_z = group["relative_influence"]

    # Create a new figure
    fig = go.Figure()

    # Add surface for this speed group
    fig.add_trace(go.Surface(
        z=z,
        x=x,
        y=y,
        colorscale="Viridis",  # Use a color scale for this surface
        showscale=True,  # Show the color scale for this plot
        name=f"RPM Left: {rpm_0}, RPM Measure: {rpm_1}"  # Title for this surface
    ))

    # Add original data points as markers
    fig.add_trace(go.Scatter3d(
        x=original_x,
        y=original_y,
        z=original_z,
        mode='markers',
        marker=dict(
            size=2,
            color='red',  # Color to mark original data points
            symbol='circle'
        ),
        name='Original Data Points'
    ))

    # Update layout
    fig.update_layout(
        title=f"Interpolated 3D Surface Plot with Original Data Points for RPM Left: {rpm_0}, RPM Measure: {rpm_1}",
        scene=dict(
            xaxis_title="Angle Measure",
            yaxis_title="Angle Left",
            zaxis_title="Relative Influence"
        ),
        template="plotly_white"
    )

    # Show the plot
    fig.show()
    # 2 d heatmap
    # overlaid slices
    # shift plot


In [ ]:
# Convert results to a DataFrame
#results_df = pd.DataFrame(results)
results_df = pd.read_csv("results.csv")

import numpy as np
import plotly.graph_objects as go

# Define the desired angle ranges for each angle
min_angle_6, max_angle_6 = -180, 180  # Range for angle_id_6 left
min_angle_7, max_angle_7 = -50, 310  # Range for angle_id_7 measure

# Function to shift angles to the desired range
def shift_angles(angle, min_angle, max_angle):
    return np.mod(angle - min_angle, max_angle - min_angle) + min_angle

# Convert angles from radians to degrees, round them, and shift to the desired ranges
results_df['angle_id_6'] = shift_angles(np.degrees(results_df['angle_id_6']).round(1), min_angle_6, max_angle_6)
results_df['angle_id_7'] = shift_angles(np.degrees(results_df['angle_id_7']).round(1), min_angle_7, max_angle_7)

# Group by speeds (max_rpm_id_0 and max_rpm_id_1)
speed_groups = results_df.groupby(["max_rpm_id_0", "max_rpm_id_1"])

# Iterate through each speed group and create a separate plot
for i, ((rpm_0, rpm_1), group) in enumerate(speed_groups):
    # Aggregate data within the group
    group = group.groupby(["angle_id_6", "angle_id_7"], as_index=False).agg({"relative_influence": "mean"})

    # Create pivot table
    pivot = group.pivot(index="angle_id_6", columns="angle_id_7", values="relative_influence")

    # Interpolate missing values
    pivot_interpolated = pivot.interpolate(method='linear', axis=0).interpolate(method='linear', axis=1)

    # Create a mask for original NaN values before interpolation
    nan_mask = pivot.isna()

    # Extract x, y, and z for the surface
    x = pivot_interpolated.columns.values  # Angle ID 7 (columns of the pivot)
    y = pivot_interpolated.index.values    # Angle ID 6 (index of the pivot)
    z = pivot_interpolated.values          # Interpolated relative influence (pivot values)

    # Ensure x, y are 2D for the surface
    x, y = np.meshgrid(x, y)

    # Identify original points
    original_x = group["angle_id_7"]
    original_y = group["angle_id_6"]
    original_z = group["relative_influence"]

    # Create a new figure
    fig = go.Figure()

    # Add surface for this speed group
    fig.add_trace(go.Surface(
        z=z,
        x=x,
        y=y,
        colorscale="Viridis",  # Use a color scale for this surface
        showscale=True,  # Show the color scale for this plot
        name=f"RPM Left: {rpm_0}, RPM Measure: {rpm_1}"  # Title for this surface
    ))

    # Add original data points as markers
    fig.add_trace(go.Scatter3d(
        x=original_x,
        y=original_y,
        z=original_z,
        mode='markers',
        marker=dict(
            size=2,
            color='red',  # Color to mark original data points
            symbol='circle'
        ),
        name='Original Data Points'
    ))

    # Update layout
    fig.update_layout(
        title=f"Interpolated 3D Surface Plot with Original Data Points for RPM Left: {rpm_0}, RPM Measure: {rpm_1}",
        scene=dict(
            xaxis_title="Angle Measure (angle_id_7)",
            yaxis_title="Angle Left (angle_id_6)",
            zaxis_title="Relative Influence"
        ),
        template="plotly_white"
    )

    # Show the plot
    fig.show()

    
    # Resample data for higher resolution
    from scipy.interpolate import griddata
    xi = np.linspace(x.min(), x.max(), 500)  # Increase resolution for x
    yi = np.linspace(y.min(), y.max(), 500)  # Increase resolution for y
    xi, yi = np.meshgrid(xi, yi)
    zi = griddata((x.flatten(), y.flatten()), z.flatten(), (xi, yi), method='cubic')


    # 2D Heatmap with higher resolution and custom colorscale
    fig_heatmap = go.Figure(data=go.Heatmap(
        z=zi,
        x=xi[0],  # Extract the first row of the meshgrid for x-axis values
        y=yi[:, 0],  # Extract the first column of the meshgrid for y-axis values
        colorscale="Cividis",  # Use a custom colorscale
        colorbar=dict(title="Relative Influence"),
    ))
    fig_heatmap.update_layout(
        title=f"High-Resolution 2D Heatmap for RPM Left: {rpm_0}, RPM Measure: {rpm_1}",
        xaxis_title="Angle Measure (angle_id_7)",
        yaxis_title="Angle Left (angle_id_6)",
        template="plotly_white"
    )
    fig_heatmap.show()


In [ ]:
#Plots relative thrust intensity

# Convert results to a DataFrame
#results_df = pd.DataFrame(results)
results_df = pd.read_csv("results.csv")

import numpy as np
import plotly.graph_objects as go

# Define the desired angle ranges for each angle
min_angle_6, max_angle_6 = -180, 180  # Range for angle_id_6
min_angle_7, max_angle_7 = -50, 310  # Range for angle_id_7

# Function to shift angles to the desired range
def shift_angles(angle, min_angle, max_angle):
    return np.mod(angle - min_angle, max_angle - min_angle) + min_angle

# Convert angles from radians to degrees, round them, and shift to the desired ranges
results_df['angle_id_6'] = shift_angles(np.degrees(results_df['angle_id_6']).round(1), min_angle_6, max_angle_6)
results_df['angle_id_7'] = shift_angles(np.degrees(results_df['angle_id_7']).round(1), min_angle_7, max_angle_7)

# Group by speeds (max_rpm_id_0 and max_rpm_id_1)
speed_groups = results_df.groupby(["max_rpm_id_0", "max_rpm_id_1"])

# Iterate through each speed group and create a separate contour plot
for i, ((rpm_0, rpm_1), group) in enumerate(speed_groups):
    # Extract the points for this group
    x = group["angle_id_7"]
    y = group["angle_id_6"]
    z = group["relative_influence"]

    # Create a 2D contour plot
    fig = go.Figure()

    # Add contour layer
    fig.add_trace(go.Contour(
        x=x,
        y=y,
        z=z,
        colorscale="jet",  # Use a high-contrast colorscale
        colorbar=dict(title="Relative Influence"),
        contours=dict(
            coloring='heatmap',  # Use a heatmap-like gradient
            showlines=True,      # Show contour lines
        ),
        name=f"Contours for RPM Left: {rpm_0}, RPM Measure: {rpm_1}"
    ))

    # Add scatter points to highlight actual data points
    #fig.add_trace(go.Scatter(
    #    x=x,
    #    y=y,
    #    mode='markers',
    #    marker=dict(
    #        size=5,
    #        color='black',
    #        symbol='circle',
    #        line=dict(width=0.5, color='white')  # Optional: outline for contrast
    #    ),
    #    name='Data Points'
    #))

    # Update layout
    fig.update_layout(
        #title=f"2D Contour Plot for RPM Left: {rpm_0}, RPM Measure: {rpm_1}",
        xaxis_title="Angle Measure (angle_id_7)",
        yaxis_title="Angle Left (angle_id_6)",
        xaxis=dict(
            scaleanchor="y",  # Link x-axis scale to y-axis scale
            title="Measuring Angle"
        ),
        yaxis=dict(
            title="Blowing Angle"
        ),
        template="plotly_white"
    )

    # Show the plot
    fig.show()


In [ ]:
# Convert results to a DataFrame
#results_df = pd.DataFrame(results)
results_df = pd.read_csv("results.csv")

import numpy as np
import plotly.graph_objects as go

# Define the desired angle ranges for each angle
min_angle_6, max_angle_6 = -180, 180  # Range for angle_id_6
min_angle_7, max_angle_7 = -50, 310  # Range for angle_id_7

# Function to shift angles to the desired range
def shift_angles(angle, min_angle, max_angle):
    return np.mod(angle - min_angle, max_angle - min_angle) + min_angle

# Convert angles from radians to degrees, round them, and shift to the desired ranges
results_df['angle_id_6'] = shift_angles(np.degrees(results_df['angle_id_6']).round(1), min_angle_6, max_angle_6)
results_df['angle_id_7'] = shift_angles(np.degrees(results_df['angle_id_7']).round(1), min_angle_7, max_angle_7)

# Group by speeds (max_rpm_id_0 and max_rpm_id_1)
speed_groups = results_df.groupby(["max_rpm_id_0", "max_rpm_id_1"])

# Iterate through each speed group and create a separate contour plot
for i, ((rpm_0, rpm_1), group) in enumerate(speed_groups):
    # Aggregate data within the group
    group = group.groupby(["angle_id_6", "angle_id_7"], as_index=False).agg({"relative_influence": "mean"})

    # Create pivot table
    pivot = group.pivot(index="angle_id_6", columns="angle_id_7", values="relative_influence")

    # Interpolate missing values
    pivot_interpolated = pivot.interpolate(method='linear', axis=0).interpolate(method='linear', axis=1)

    # Extract x, y, and z for the surface
    x = pivot_interpolated.columns.values  # Angle ID 7 (columns of the pivot)
    y = pivot_interpolated.index.values    # Angle ID 6 (index of the pivot)
    z = pivot_interpolated.values          # Interpolated relative influence (pivot values)

    # Ensure x, y are 2D for the surface
    x, y = np.meshgrid(x, y)

    # Create a 2D contour plot
    fig = go.Figure()

    
    
    ## Add contour layer
    #fig.add_trace(go.Contour(
    #    x=x,
    #    y=y,
    #    z=z,
    #    colorscale="jet",  # Use a high-contrast colorscale
    #    colorbar=dict(title="Relative Influence"),
    #    contours=dict(
    #        coloring='heatmap',  # Use a heatmap-like gradient
    #        showlines=True,      # Show contour lines
    #    ),
    #    name=f"Contours for RPM Left: {rpm_0}, RPM Measure: {rpm_1}"
    #))

    # Add contour plot as the baseline
    fig.add_trace(go.Contour(
        z=z,
        x=pivot_interpolated.columns.values,
        y=pivot_interpolated.index.values,
        colorscale="jet",
        colorbar=dict(title="Relative Influence"),
        contours=dict(showlabels=True,
                      coloring='heatmap'),
        name="Contour Baseline"
    ))

    # Add scatter points to highlight actual data points
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers',
        marker=dict(
            size=5,
            color='black',
            symbol='circle',
            line=dict(width=0.5, color='white')  # Optional: outline for contrast
        ),
        name='Data Points'
    ))

    # Update layout
    fig.update_layout(
        #title=f"Contour Plot with Relative Influence",
        xaxis_title="Angle Measure", #(angle_id_7)
        yaxis_title="Angle Left", # (angle_id_6)
        xaxis=dict(
            scaleanchor="y",  # Link x-axis scale to y-axis scale
            title="Measuring Angle" # (angle_id_7)
        ),
        yaxis=dict(
            title="Blowing Angle" # (angle_id_6)
        ),
        template="plotly_white"
    )

    # Show the plot
    fig.show()


In [30]:
def calculate_stds_for_groups_with_labels(data, time_data, groups, column_idx, angle_data, rpm_data):
    """
    Calculates the standard deviation of data for each group of 3 pairs of time points, 
    labeling each group with constant angles and maximum speeds.

    Args:
        data (np.ndarray): Data array to analyze.
        time_data (np.ndarray): Time series corresponding to the data.
        groups (list of list of tuples): Groups of 3 pairs of time points.
                                          Each group is a list of tuples [(t1_start, t1_end), ...].
        column_idx (int): Column index in the data array to analyze.
        angle_data (np.ndarray): Array of angle data (e.g., `id_6` and `id_7` for angles).
        rpm_data (np.ndarray): Array of RPM data (e.g., `id_0` and `id_1` for speeds).

    Returns:
        list: Results for each group as a dictionary.
    """
    results_std = []

    for group_idx, group in enumerate(groups):
        if len(group) != 3:
            print(f"Skipping group {group_idx + 1}: Not exactly 3 pairs.")
            continue

        # Use the angles from the first pair of the group
        first_pair_mask = (time_data >= group[0][0]) & (time_data <= group[0][1])
        angle_id_6 = angle_data[first_pair_mask, 0][0]  # Take first value of id_6
        angle_id_7 = angle_data[first_pair_mask, 1][0]  # Take first value of id_7

        # Calculate the max RPMs for the entire group
        group_mask = np.any([(time_data > start) & (time_data < end) for start, end in group], axis=0)
        max_rpm_id_0 = np.max(rpm_data[group_mask, 0])  # Max RPM (id_0)
        max_rpm_id_1 = np.max(rpm_data[group_mask, 1])  # Max RPM (id_1)

        stds = []
        for start, end in group:
            # Mask to select data between start and end times
            mask = (time_data >= start) & (time_data <= end)
            selected_data = data[mask, column_idx]

            # Calculate standard deviation and handle empty selection gracefully
            std_value = np.std(selected_data) if len(selected_data) > 0 else np.nan
            stds.append(std_value)

        if len(stds) == 3:
            influence_std, baseline_std, bias_std = stds
            baseline_corrected_std = baseline_std - bias_std
            influence_corrected_std = influence_std - bias_std
            absolute_influence_std = influence_corrected_std - baseline_corrected_std
            relative_influence_std = (absolute_influence_std / baseline_corrected_std) * 100 if baseline_corrected_std != 0 else np.nan

            results_std.append({
                "group": group_idx + 1,
                "angle_id_6": angle_id_6,
                "angle_id_7": angle_id_7,
                "max_rpm_id_0": max_rpm_id_0,
                "max_rpm_id_1": max_rpm_id_1,
                "influence_std": influence_std,
                "baseline_std": baseline_std,
                "bias_std": bias_std,
                "baseline_corrected_std": baseline_corrected_std,
                "influence_corrected_std": influence_corrected_std,
                "absolute_influence_std": absolute_influence_std,
                "relative_influence_std": relative_influence_std,
            })

    return results_std


# Example Usage
# Assuming `data_array`, `time_data`, `angle_data`, `rpm_data`, and `groups` are defined
column_idx_to_analyze = 1
rpm_data = data_array[:, 2:4]
angle_data = data_array[:, 4:]
results_std = calculate_stds_for_groups_with_labels(
    data_array, time_data, time_groups, column_idx_to_analyze, angle_data, rpm_data
)

# Convert results to a DataFrame
results_std_df = pd.DataFrame(results_std)


In [ ]:
# Convert results to a DataFrame
#results_df = pd.DataFrame(results)
results_df = pd.read_csv("results.csv")

import numpy as np
import plotly.graph_objects as go

# Define the desired angle ranges for each angle
min_angle_6, max_angle_6 = -180, 180  # Range for angle_id_6
min_angle_7, max_angle_7 = -50, 310  # Range for angle_id_7

# Function to shift angles to the desired range
def shift_angles(angle, min_angle, max_angle):
    return np.mod(angle - min_angle, max_angle - min_angle) + min_angle

# Convert angles from radians to degrees, round them, and shift to the desired ranges
results_std_df['angle_id_6'] = shift_angles(np.degrees(results_std_df['angle_id_6']).round(1), min_angle_6, max_angle_6)
results_std_df['angle_id_7'] = shift_angles(np.degrees(results_std_df['angle_id_7']).round(1), min_angle_7, max_angle_7)

# Group by speeds (max_rpm_id_0 and max_rpm_id_1)
speed_groups = results_std_df.groupby(["max_rpm_id_0", "max_rpm_id_1"])

# Iterate through each speed group and create a separate contour plot
for i, ((rpm_0, rpm_1), group) in enumerate(speed_groups):
    # Aggregate data within the group
    group = group.groupby(["angle_id_6", "angle_id_7"], as_index=False).agg({"influence_std": "mean"})

    # Create pivot table
    pivot = group.pivot(index="angle_id_6", columns="angle_id_7", values="influence_std")

    # Interpolate missing values
    pivot_interpolated = pivot.interpolate(method='linear', axis=0).interpolate(method='linear', axis=1)

    # Extract x, y, and z for the surface
    x = pivot_interpolated.columns.values  # Angle ID 7 (columns of the pivot)
    y = pivot_interpolated.index.values    # Angle ID 6 (index of the pivot)
    z = pivot_interpolated.values          # Interpolated relative influence (pivot values)

    # Ensure x, y are 2D for the surface
    x, y = np.meshgrid(x, y)

    # Create a 2D contour plot
    fig = go.Figure()

    
    
    ## Add contour layer
    #fig.add_trace(go.Contour(
    #    x=x,
    #    y=y,
    #    z=z,
    #    colorscale="jet",  # Use a high-contrast colorscale
    #    colorbar=dict(title="Relative Influence"),
    #    contours=dict(
    #        coloring='heatmap',  # Use a heatmap-like gradient
    #        showlines=True,      # Show contour lines
    #    ),
    #    name=f"Contours for RPM Left: {rpm_0}, RPM Measure: {rpm_1}"
    #))

    # Add contour plot as the baseline
    fig.add_trace(go.Contour(
        z=z,
        x=pivot_interpolated.columns.values,
        y=pivot_interpolated.index.values,
        colorscale="jet",
        colorbar=dict(title="Standard Deviation"),
        contours=dict(showlabels=True,
                      coloring='heatmap'),
        name="Contour Baseline"
    ))

    # Add scatter points to highlight actual data points
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers',
        marker=dict(
            size=5,
            color='black',
            symbol='circle',
            line=dict(width=0.5, color='white')  # Optional: outline for contrast
        ),
        name='Data Points'
    ))

    # Update layout
    fig.update_layout(
        title=f"Contour Plot with Standard Deviation",
        xaxis_title="Angle Measure", #(angle_id_7)
        yaxis_title="Angle Left", # (angle_id_6)
        xaxis=dict(
            scaleanchor="y",  # Link x-axis scale to y-axis scale
            title="Angle Measure" # (angle_id_7)
        ),
        yaxis=dict(
            title="Angle Left" # (angle_id_6)
        ),
        template="plotly_white"
    )

    # Show the plot
    fig.show()


In [ ]:
# Convert results to a DataFrame
#results_df = pd.DataFrame(results)
results_df = pd.read_csv("results.csv")

import numpy as np
import plotly.graph_objects as go

# Group by speeds (max_rpm_id_0 and max_rpm_id_1)
speed_groups = results_df.groupby(["max_rpm_id_0", "max_rpm_id_1"])

# Convert angles from radians to degrees and round them
results_df['angle_id_6'] = np.degrees(results_df['angle_id_6']).round(1)  # Rounded to 1 decimal place
results_df['angle_id_7'] = np.degrees(results_df['angle_id_7']).round(1)

# Number of largest gradients to display
top_n_gradients = 10

# Iterate through each speed group and create a separate plot
for i, ((rpm_0, rpm_1), group) in enumerate(speed_groups):
    # Aggregate data within the group
    group = group.groupby(["angle_id_6", "angle_id_7"], as_index=False).agg({"relative_influence": "mean"})

    # Create pivot table
    pivot = group.pivot(index="angle_id_6", columns="angle_id_7", values="relative_influence")

    # Interpolate missing values
    pivot_interpolated = pivot.interpolate(method='linear', axis=0).interpolate(method='linear', axis=1)

    # Extract x, y, and z for the surface
    x = pivot_interpolated.columns.values  # Angle ID 7 (columns of the pivot)
    y = pivot_interpolated.index.values    # Angle ID 6 (index of the pivot)
    z = pivot_interpolated.values          # Interpolated relative influence (pivot values)

    # Ensure x, y are 2D for the surface
    x, y = np.meshgrid(x, y)

    # Calculate gradients
    dz_dx, dz_dy = np.gradient(z, axis=(1, 0))  # Gradients along x and y
    gradient_magnitude = np.sqrt(dz_dx**2 + dz_dy**2)  # Gradient magnitude

    # Flatten the gradient data to sort and select top N gradients
    grad_flat_indices = np.argsort(gradient_magnitude.ravel())[::-1][:top_n_gradients]
    grad_y, grad_x = np.unravel_index(grad_flat_indices, gradient_magnitude.shape)

    # Extract the coordinates and gradient vectors for the largest gradients
    grad_positions_x = x[grad_y, grad_x]
    grad_positions_y = y[grad_y, grad_x]
    grad_positions_z = z[grad_y, grad_x]
    grad_vectors_dx = dz_dx[grad_y, grad_x]
    grad_vectors_dy = dz_dy[grad_y, grad_x]

    # Create a new figure
    fig = go.Figure()

    # Add contour plot as the baseline
    fig.add_trace(go.Contour(
        z=z,
        x=pivot_interpolated.columns.values,
        y=pivot_interpolated.index.values,
        colorscale="Viridis",
        contours=dict(showlabels=True),
        name="Contour Baseline"
    ))

    # Add markers for largest gradients
    fig.add_trace(go.Scatter3d(
        x=grad_positions_x,
        y=grad_positions_y,
        z=grad_positions_z,
        mode='markers',
        marker=dict(
            size=6,
            color='blue',  # Use a distinct color for gradient markers
            symbol='circle'
        ),
        name='Largest Gradient Points'
    ))

    # Add arrows to visualize direction and magnitude of gradients
    for gx, gy, gz, dx, dy in zip(grad_positions_x, grad_positions_y, grad_positions_z, grad_vectors_dx, grad_vectors_dy):
        fig.add_trace(go.Scatter3d(
            x=[gx, gx + dx],
            y=[gy, gy + dy],
            z=[gz, gz],  # Gradients are visualized as horizontal vectors
            mode='lines',
            line=dict(color='red', width=3),
            name='Gradient Vector'
        ))

    # Update layout
    fig.update_layout(
        title=f"Contour Plot with Largest Gradients for RPM Left: {rpm_0}, RPM Measure: {rpm_1}",
        scene=dict(
            xaxis_title="Angle Measure (angle_id_7)",
            yaxis_title="Angle Left (angle_id_6)",
            zaxis_title="Relative Influence"
        ),
        template="plotly_white"
    )

    # Show the plot
    fig.show()


In [ ]:
# Convert results to a DataFrame
#results_df = pd.DataFrame(results)
results_df = pd.read_csv("results.csv")

import numpy as np
import plotly.graph_objects as go

# Define the desired angle ranges for each angle
min_angle_6, max_angle_6 = -180, 180  # Range for angle_id_6
min_angle_7, max_angle_7 = -50, 210  # Range for angle_id_7

# Function to shift angles to the desired range
def shift_angles(angle, min_angle, max_angle):
    return np.mod(angle - min_angle, max_angle - min_angle) + min_angle

# Convert angles from radians to degrees, round them, and shift to the desired ranges
results_df['angle_id_6'] = shift_angles(np.degrees(results_df['angle_id_6']).round(1), min_angle_6, max_angle_6)
results_df['angle_id_7'] = shift_angles(np.degrees(results_df['angle_id_7']).round(1), min_angle_7, max_angle_7)

# Group by speeds (max_rpm_id_0 and max_rpm_id_1)
speed_groups = results_df.groupby(["max_rpm_id_0", "max_rpm_id_1"])

# Iterate through each speed group and create a gradient plot
for i, ((rpm_0, rpm_1), group) in enumerate(speed_groups):
    # Create pivot table for gradients
    pivot = group.pivot(index="angle_id_6", columns="angle_id_7", values="relative_influence")

    # Interpolate missing values
    pivot_interpolated = pivot.interpolate(method='linear', axis=0).interpolate(method='linear', axis=1)

    # Compute gradients
    x = pivot_interpolated.columns.values  # angle_id_7
    y = pivot_interpolated.index.values    # angle_id_6
    z = pivot_interpolated.values          # relative_influence values

    # Compute gradients
    grad_y, grad_x = np.gradient(z, edge_order=2)  # Compute gradients along y and x
    grad_magnitude = np.sqrt(grad_x**2 + grad_y**2)  # Gradient magnitude

    # Ensure x, y are 2D for quiver plot
    x, y = np.meshgrid(x, y)

    # Create a new figure for gradient visualization
    fig = go.Figure()

    # Add magnitude as a heatmap
    fig.add_trace(go.Heatmap(
        x=pivot_interpolated.columns,
        y=pivot_interpolated.index,
        z=grad_magnitude,
        colorscale="Turbo",
        colorbar=dict(title="Gradient Magnitude"),
        name="Gradient Magnitude"
    ))

    # Add quiver plot for direction
    fig.add_trace(go.Scatter(
        x=x.flatten(),
        y=y.flatten(),
        mode='markers',
        marker=dict(size=1, color='black'),
        name="Gradient Direction"
    ))
    for i in range(0, len(y), 3):
        #X0 + Right Amount


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Convert results to a DataFrame
#results_df = pd.DataFrame(results)
results_df = pd.read_csv("results.csv")

# Group by speeds (max_rpm_id_0 and max_rpm_id_1)
speed_groups = results_df.groupby(["max_rpm_id_0", "max_rpm_id_1"])

# Convert angles from radians to degrees and round them
results_df['angle_id_6'] = np.degrees(results_df['angle_id_6']).round(1)  # Rounded to 1 decimal place
results_df['angle_id_7'] = np.degrees(results_df['angle_id_7']).round(1)

# Specify slices for 2D plots
specific_y_values = [-90]  # Replace with desired angle_id_6 values
specific_x_values = [None]  # Replace with desired angle_id_7 values

# Iterate through each speed group and create plots
for i, ((rpm_0, rpm_1), group) in enumerate(speed_groups):
    # Aggregate data within the group
    group = group.groupby(["angle_id_6", "angle_id_7"], as_index=False).agg({"relative_influence": "mean"})

    # Create pivot table
    pivot = group.pivot(index="angle_id_6", columns="angle_id_7", values="relative_influence")

    # Interpolate missing values
    pivot_interpolated = pivot.interpolate(method='linear', axis=0).interpolate(method='linear', axis=1)

    # Create a mask for original NaN values before interpolation
    nan_mask = pivot.isna()

    # Extract x, y, and z for the surface
    x = pivot_interpolated.columns.values  # Angle ID 7 (columns of the pivot)
    y = pivot_interpolated.index.values    # Angle ID 6 (index of the pivot)
    z = pivot_interpolated.values          # Interpolated relative influence (pivot values)

    # Ensure x, y are 2D for the surface
    x, y = np.meshgrid(x, y)

    # Create a new figure
    fig = go.Figure()

    # Add surface for this speed group
    fig.add_trace(go.Surface(
        z=z,
        x=x,
        y=y,
        colorscale="Viridis",  # Use a color scale for this surface
        showscale=True,  # Show the color scale for this plot
        name=f"RPM Left: {rpm_0}, RPM Measure: {rpm_1}"  # Title for this surface
    ))

    # Update layout
    fig.update_layout(
        title=f"Interpolated 3D Surface Plot for RPM Left: {rpm_0}, RPM Measure: {rpm_1}",
        scene=dict(
            xaxis_title="Angle Measure (angle_id_7)",
            yaxis_title="Angle Left (angle_id_6)",
            zaxis_title="Relative Influence"
        ),
        template="plotly_white"
    )

    # Show the 3D plot
    fig.show()

    # 2D Slices at specific y values
    for specific_y_value in specific_y_values:
        if specific_y_value in y:
            y_index = np.argmin(np.abs(y[:, 0] - specific_y_value))
            slice_x = x[y_index, :]
            slice_z = z[y_index, :]

            # Create a 2D line plot for the slice
            fig_2d_y = go.Figure()
            fig_2d_y.add_trace(go.Scatter(
                x=slice_x,
                y=slice_z,
                mode="lines+markers",
                name=f"Slice at Angle Left = {specific_y_value}"
            ))
            fig_2d_y.update_layout(
                title=f"2D Slice at Angle Left = {specific_y_value} for RPM Measure: {rpm_1}",
                xaxis_title="Angle Measure (angle_id_7)",
                yaxis_title="Relative Influence",
                template="plotly_white"
            )
            fig_2d_y.show()

    # 2D Slices at specific x values
    for specific_x_value in specific_x_values:
        if specific_x_value in x[0, :]:
            x_index = np.argmin(np.abs(x[0, :] - specific_x_value))
            slice_y = y[:, x_index]
            slice_z = z[:, x_index]

            # Create a 2D line plot for the slice
            fig_2d_x = go.Figure()
            fig_2d_x.add_trace(go.Scatter(
                x=slice_y,
                y=slice_z,
                mode="lines+markers",
                name=f"Slice at Angle Measure = {specific_x_value}"
            ))
            fig_2d_x.update_layout(
                title=f"2D Slice at Angle Measure = {specific_x_value} for RPM Measure: {rpm_1}",
                xaxis_title="Angle Left (angle_id_6)",
                yaxis_title="Relative Influence",
                template="plotly_white"
            )
            fig_2d_x.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

# Group by speeds (max_rpm_id_0 and max_rpm_id_1)
speed_groups = results_df.groupby(["max_rpm_id_0", "max_rpm_id_1"])

# Iterate through each speed group and create a separate plot
for i, ((rpm_0, rpm_1), group) in enumerate(speed_groups):
    # Create pivot table
    pivot = group.pivot(index="angle_id_6", columns="angle_id_7", values="absolute_influence")

    # Extract x, y, and z for the surface
    x = pivot.columns.values  # Angle ID 7 (columns of the pivot)
    y = pivot.index.values    # Angle ID 6 (index of the pivot)
    z = pivot.values          # Relative influence (pivot values)

    # Ensure x, y are 2D for the surface
    x, y = np.meshgrid(x, y)

    # Create a new figure
    fig = go.Figure()

    # Add surface for this speed group
    fig.add_trace(go.Surface(
        z=z,
        x=x,
        y=y,
        colorscale="Viridis",  # Use a color scale for this surface
        showscale=True,  # Show the color scale for this plot
        name=f"RPM 0: {rpm_0}, RPM 1: {rpm_1}"  # Title for this surface
    ))

    # Update layout
    fig.update_layout(
        title=f"3D Surface Plot for RPM Left: {rpm_0}, RPM Measure: {rpm_1}",
        scene=dict(
            xaxis_title="Angle Measure",
            yaxis_title="Angle Left",
            zaxis_title="Absolute Influence"
        ),
        template="plotly_white"
    )

    # Show the plot
    fig.show()


In [16]:
# Get unique speed combinations for both motors
speed_combinations = results_df[["max_rpm_id_0", "max_rpm_id_1"]].drop_duplicates()

# Create an empty list to store results for each speed combination
all_averaged_results = []

# Iterate over each speed combination
for _, (rpm_0, rpm_1) in speed_combinations.iterrows():
    # Filter data for the current speed combination
    filtered_results = results_df[
        (results_df["max_rpm_id_0"] == rpm_0) & (results_df["max_rpm_id_1"] == rpm_1)
    ]

    # Group by angles (angle_id_6 and angle_id_7) and compute averages
    averaged_results = filtered_results.groupby(["angle_id_6", "angle_id_7"])[
        ["absolute_influence", "relative_influence"]
    ].mean().reset_index()

    # Add the speed combination to the averaged results
    averaged_results["max_rpm_id_0"] = rpm_0
    averaged_results["max_rpm_id_1"] = rpm_1

    # Append the averaged results to the list
    all_averaged_results.append(averaged_results)

# Combine all results into a single DataFrame
all_averaged_results_df = pd.concat(all_averaged_results, ignore_index=True)

# Rename columns for clarity
all_averaged_results_df.rename(columns={
    "absolute_influence": "avg_absolute_influence",
    "relative_influence": "avg_relative_influence"
}, inplace=True)



In [17]:
# Group by rpm1 (max_rpm_id_1) and angles (angle_id_6, angle_id_7)
averaged_over_rpm1 = results_df.groupby(["max_rpm_id_1", "angle_id_6", "angle_id_7"])[
    ["absolute_influence", "relative_influence"]
].mean().reset_index()

# Rename columns for clarity
averaged_over_rpm1.rename(columns={
    "absolute_influence": "avg_absolute_influence",
    "relative_influence": "avg_relative_influence"
}, inplace=True)


In [ ]:
import numpy as np
import plotly.graph_objects as go

# Iterate over each unique `max_rpm_id_1` value
unique_rpm0_values = averaged_over_rpm1["max_rpm_id_1"].unique()

for rpm_1 in unique_rpm0_values:
    # Filter the averaged results for the current `max_rpm_id_1` value
    filtered_results = averaged_over_rpm1[averaged_over_rpm1["max_rpm_id_1"] == rpm_1]

    # Create pivot table for plotting
    pivot = filtered_results.pivot(index="angle_id_6", columns="angle_id_7", values="avg_relative_influence")

    # Extract x, y, and z for the surface
    x = pivot.columns.values  # Angle ID 7 (columns of the pivot)
    y = pivot.index.values    # Angle ID 6 (index of the pivot)
    z = pivot.values          # Average relative influence (pivot values)

    # Ensure x, y are 2D for the surface
    x, y = np.meshgrid(x, y)

    # Create a new figure
    fig = go.Figure()

    # Add surface for this motor speed
    fig.add_trace(go.Surface(
        z=z,
        x=x,
        y=y,
        colorscale="Viridis",  # Use a color scale for this surface
        showscale=True,  # Show the color scale
        name=f"RPM 1: {rpm_0}"  # Title for this surface
    ))

    # Update layout
    fig.update_layout(
        title=f"3D Surface Plot for RPM Measure: {rpm_1}",
        scene=dict(
            xaxis_title="Angle ID 7",
            yaxis_title="Angle ID 6",
            zaxis_title="Avg Relative Influence"
        ),
        template="plotly_white"
    )

    # Show the plot
    fig.show()


In [19]:
# Group by rpm1 (max_rpm_id_0) and angles (angle_id_6, angle_id_7)
averaged_over_rpm0 = results_df.groupby(["max_rpm_id_0", "angle_id_6", "angle_id_7"])[
    ["absolute_influence", "relative_influence"]
].mean().reset_index()

# Rename columns for clarity
averaged_over_rpm0.rename(columns={
    "absolute_influence": "avg_absolute_influence",
    "relative_influence": "avg_relative_influence"
}, inplace=True)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Iterate over each unique `max_rpm_id_0` value
unique_rpm0_values = averaged_over_rpm0["max_rpm_id_0"].unique()

for rpm_0 in unique_rpm0_values:
    # Filter the averaged results for the current `max_rpm_id_0` value
    filtered_results = averaged_over_rpm0[averaged_over_rpm0["max_rpm_id_0"] == rpm_0]

    # Create pivot table for plotting
    pivot = filtered_results.pivot(index="angle_id_6", columns="angle_id_7", values="avg_relative_influence")

    # Extract x, y, and z for the surface
    x = pivot.columns.values  # Angle ID 7 (columns of the pivot)
    y = pivot.index.values    # Angle ID 6 (index of the pivot)
    z = pivot.values          # Average relative influence (pivot values)

    # Ensure x, y are 2D for the surface
    x, y = np.meshgrid(x, y)

    # Create a new figure
    fig = go.Figure()

    # Add surface for this motor speed
    fig.add_trace(go.Surface(
        z=z,
        x=x,
        y=y,
        colorscale="Viridis",  # Use a color scale for this surface
        showscale=True,  # Show the color scale
        name=f"RPM 1: {rpm_0}"  # Title for this surface
    ))

    # Update layout
    fig.update_layout(
        title=f"3D Surface Plot for RPM Left: {rpm_0}",
        scene=dict(
            xaxis_title="Angle ID 7",
            yaxis_title="Angle ID 6",
            zaxis_title="Avg Relative Influence"
        ),
        template="plotly_white"
    )

    # Show the plot
    fig.show()


In [21]:
sym_results_df = results_df.copy()
import numpy as np
import pandas as pd

# Convert angles from radians to degrees
sym_results_df["angle_id_6"] = np.degrees(sym_results_df["angle_id_6"])
sym_results_df["angle_id_7"] = np.degrees(sym_results_df["angle_id_7"])

# Create a copy of the DataFrame for the transformed angles
df_transformed = sym_results_df.copy()

# Ensure all angles are non-negative
sym_results_df["angle_id_6"] = np.where(sym_results_df["angle_id_6"] < 0, sym_results_df["angle_id_6"] + 360, sym_results_df["angle_id_6"])
sym_results_df["angle_id_7"] = np.where(sym_results_df["angle_id_7"] < 0, sym_results_df["angle_id_7"] + 360, sym_results_df["angle_id_7"])

# Apply the angle transformation using NumPy
df_transformed["angle_id_6"] = (-df_transformed["angle_id_6"] + 180) % 360
df_transformed["angle_id_7"] = (-df_transformed["angle_id_7"] + 180) % 360



#print(sym_results_df[["angle_id_6", "angle_id_7"]].to_string())
#print(df_transformed[["angle_id_6", "angle_id_7"]].to_string())



# Adjust the `group` column in the transformed DataFrame to ensure unique group numbers
max_group = sym_results_df["group"].max()  # Find the highest existing group number
df_transformed["group"] += max_group  # Increment group numbers in the transformed DataFrame

# Concatenate the original and transformed DataFrames
df_combined = pd.concat([sym_results_df, df_transformed], ignore_index=True)

df_combined["angle_id_6"] = df_combined["angle_id_6"].round(1)
df_combined["angle_id_7"] = df_combined["angle_id_7"].round(1)


#print(df_combined.shape)
## Aggregate duplicate rows by averaging the relative_influence
#df_combined = df_combined.groupby(
#    ["group", "angle_id_6", "angle_id_7", "max_rpm_id_0", "max_rpm_id_1"],
#    as_index=False
#).agg({"relative_influence": "mean"})
#print(df_combined.shape)

# Sort the final DataFrame by `max_rpm_id_0`, `max_rpm_id_1`, and angles
df_combined.sort_values(by=["angle_id_7", "angle_id_6", "max_rpm_id_1", "max_rpm_id_0"], inplace=True)

# Reset the index after sorting
df_combined.reset_index(drop=True, inplace=True)




In [ ]:
print(df_combined.to_string())

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Group by speeds (max_rpm_id_0 and max_rpm_id_1)
speed_groups = df_combined.groupby(["max_rpm_id_0", "max_rpm_id_1"])

# Iterate through each speed group and create a separate plot
for i, ((rpm_0, rpm_1), group) in enumerate(speed_groups):
    # Create pivot table
    group = group.groupby(["angle_id_6", "angle_id_7"], as_index=False).agg({"relative_influence": "mean"})

    # Now perform the pivot operation
    pivot = group.pivot(index="angle_id_6", columns="angle_id_7", values="relative_influence")

    # Extract x, y, and z for the surface
    x = pivot.columns.values  # Angle ID 7 (columns of the pivot)
    y = pivot.index.values    # Angle ID 6 (index of the pivot)
    z = pivot.values          # Relative influence (pivot values)

    # Ensure x, y are 2D for the surface
    x, y = np.meshgrid(x, y)

    # Create a new figure
    fig = go.Figure()

    # Add surface for this speed group
    fig.add_trace(go.Surface(
        z=z,
        x=x,
        y=y,
        colorscale="Viridis",  # Use a color scale for this surface
        showscale=True,  # Show the color scale for this plot
        name=f"RPM 0: {rpm_0}, RPM 1: {rpm_1}"  # Title for this surface
    ))

    # Update layout
    fig.update_layout(
        title=f"3D Surface Plot for RPM Left: {rpm_0}, RPM Measure: {rpm_1}",
        scene=dict(
            xaxis_title="Angle Measure",
            yaxis_title="Angle Left",
            zaxis_title="Relative Influence"
        ),
        template="plotly_white"
    )

    # Show the plot
    fig.show()


In [170]:
# Group by rpm1 (max_rpm_id_1) and angles (angle_id_6, angle_id_7)
averaged_over_rpm1 = df_combined.groupby(["max_rpm_id_1", "angle_id_6", "angle_id_7"])[
    ["absolute_influence", "relative_influence"]
].mean().reset_index()

# Rename columns for clarity
averaged_over_rpm1.rename(columns={
    "absolute_influence": "avg_absolute_influence",
    "relative_influence": "avg_relative_influence"
}, inplace=True)


In [ ]:
import numpy as np
import plotly.graph_objects as go

# Iterate over each unique `max_rpm_id_1` value
unique_rpm1_values = averaged_over_rpm1["max_rpm_id_1"].unique()

for rpm_1 in unique_rpm1_values:
    # Filter the averaged results for the current `max_rpm_id_1` value
    filtered_results = averaged_over_rpm1[averaged_over_rpm1["max_rpm_id_1"] == rpm_1]

    # Create pivot table for plotting
    pivot = filtered_results.pivot(index="angle_id_6", columns="angle_id_7", values="avg_relative_influence")

    # Extract x, y, and z for the surface
    x = pivot.columns.values  # Angle ID 7 (columns of the pivot)
    y = pivot.index.values    # Angle ID 6 (index of the pivot)
    z = pivot.values          # Average relative influence (pivot values)

    # Ensure x, y are 2D for the surface
    x, y = np.meshgrid(x, y)

    # Create a new figure
    fig = go.Figure()

    # Add surface for this motor speed
    fig.add_trace(go.Surface(
        z=z,
        x=x,
        y=y,
        colorscale="Viridis",  # Use a color scale for this surface
        showscale=True,  # Show the color scale
        name=f"RPM 1: {rpm_0}"  # Title for this surface
    ))

    # Update layout
    fig.update_layout(
        title=f"3D Surface Plot for RPM Measure: {rpm_1}",
        scene=dict(
            xaxis_title="Angle ID 7",
            yaxis_title="Angle ID 6",
            zaxis_title="Avg Relative Influence"
        ),
        template="plotly_white"
    )

    # Show the plot
    fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

# Specify the list of values for x (angle_id_7) or y (angle_id_6) for the slices
specific_y_values = [255, 270, 285]  # Replace with the desired angle_id_6 values
specific_x_values = [None]  # Replace with the desired angle_id_7 values

# Iterate over each unique `max_rpm_id_1` value
unique_rpm1_values = averaged_over_rpm1["max_rpm_id_1"].unique()

for rpm_1 in unique_rpm1_values:
    # Filter the averaged results for the current `max_rpm_id_1` value
    filtered_results = averaged_over_rpm1[averaged_over_rpm1["max_rpm_id_1"] == rpm_1]

    # Create pivot table for plotting
    pivot = filtered_results.pivot(index="angle_id_6", columns="angle_id_7", values="avg_relative_influence")

    # Extract x, y, and z for the surface
    x = pivot.columns.values  # Angle ID 7 (columns of the pivot)
    y = pivot.index.values    # Angle ID 6 (index of the pivot)
    z = pivot.values          # Average relative influence (pivot values)

    # Ensure x, y are 2D for the surface
    x, y = np.meshgrid(x, y)

    # Create a new figure
    fig = go.Figure()

    # Add surface for this motor speed
    fig.add_trace(go.Surface(
        z=z,
        x=x,
        y=y,
        colorscale="Viridis",  # Use a color scale for this surface
        showscale=True,  # Show the color scale
        name=f"RPM 1: {rpm_1}"  # Title for this surface
    ))

    # Update layout
    fig.update_layout(
        title=f"3D Surface Plot for RPM Measure: {rpm_1}",
        scene=dict(
            xaxis_title="Angle Measure (angle_id_7)",
            yaxis_title="Angle Left (angle_id_6)",
            zaxis_title="Avg Relative Influence"
        ),
        template="plotly_white"
    )

    # Show the plot
    fig.show()

    # Create 2D slices at specific y values
    for specific_y_value in specific_y_values:
        if specific_y_value in pivot.index:
            slice_data = pivot.loc[specific_y_value]
            slice_x = slice_data.index
            slice_z = slice_data.values

            # Create a 2D line plot for the slice
            fig_2d_y = go.Figure()
            fig_2d_y.add_trace(go.Scatter(
                x=slice_x,
                y=slice_z,
                mode="lines+markers",
                name=f"Slice at Left = {specific_y_value}"
            ))
            fig_2d_y.update_layout(
                title=f"2D Slice at Left = {specific_y_value} for RPM Measure: {rpm_1}",
                xaxis_title="Angle Measure (angle_id_7)",
                yaxis_title="Avg Relative Influence",
                template="plotly_white"
            )
            fig_2d_y.show()

    # Create 2D slices at specific x values
    for specific_x_value in specific_x_values:
        if specific_x_value in pivot.columns:
            slice_data = pivot[specific_x_value]
            slice_y = slice_data.index
            slice_z = slice_data.values

            # Create a 2D line plot for the slice
            fig_2d_x = go.Figure()
            fig_2d_x.add_trace(go.Scatter(
                x=slice_y,
                y=slice_z,
                mode="lines+markers",
                name=f"Slice at angle_id_7 = {specific_x_value}"
            ))
            fig_2d_x.update_layout(
                title=f"2D Slice at angle_id_7 = {specific_x_value} for RPM Measure: {rpm_1}",
                xaxis_title="Angle Left (angle_id_6)",
                yaxis_title="Avg Relative Influence",
                template="plotly_white"
            )
            fig_2d_x.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

# Specify the list of values for x (angle_id_7) or y (angle_id_6) for the slices
specific_y_values = [255, 270, 285]  # Replace with the desired angle_id_6 values
specific_x_values = [None]  # Replace with the desired angle_id_7 values

# Iterate over each unique `max_rpm_id_1` value
unique_rpm1_values = averaged_over_rpm1["max_rpm_id_1"].unique()

for rpm_1 in unique_rpm1_values:
    # Filter the averaged results for the current `max_rpm_id_1` value
    filtered_results = averaged_over_rpm1[averaged_over_rpm1["max_rpm_id_1"] == rpm_1]

    # Create pivot table for plotting
    pivot = filtered_results.pivot(index="angle_id_6", columns="angle_id_7", values="avg_absolute_influence")

    # Extract x, y, and z for the surface
    x = pivot.columns.values  # Angle ID 7 (columns of the pivot)
    y = pivot.index.values    # Angle ID 6 (index of the pivot)
    z = pivot.values          # Average relative influence (pivot values)

    # Ensure x, y are 2D for the surface
    x, y = np.meshgrid(x, y)

    # Create a new figure
    fig = go.Figure()

    # Add surface for this motor speed
    fig.add_trace(go.Surface(
        z=z,
        x=x,
        y=y,
        colorscale="Viridis",  # Use a color scale for this surface
        showscale=True,  # Show the color scale
        name=f"RPM 1: {rpm_1}"  # Title for this surface
    ))

    # Update layout
    fig.update_layout(
        title=f"3D Surface Plot for RPM Measure: {rpm_1}",
        scene=dict(
            xaxis_title="Angle Measure (angle_id_7)",
            yaxis_title="Angle Left (angle_id_6)",
            zaxis_title="Avg Absolute Influence"
        ),
        template="plotly_white"
    )

    # Show the plot
    fig.show()

    # Create 2D slices at specific y values
    for specific_y_value in specific_y_values:
        if specific_y_value in pivot.index:
            slice_data = pivot.loc[specific_y_value]
            slice_x = slice_data.index
            slice_z = slice_data.values

            # Create a 2D line plot for the slice
            fig_2d_y = go.Figure()
            fig_2d_y.add_trace(go.Scatter(
                x=slice_x,
                y=slice_z,
                mode="lines+markers",
                name=f"Slice at Left = {specific_y_value}"
            ))
            fig_2d_y.update_layout(
                title=f"2D Slice at Left = {specific_y_value} for RPM Measure: {rpm_1}",
                xaxis_title="Angle Measure (angle_id_7)",
                yaxis_title="Avg Absolute Influence",
                template="plotly_white"
            )
            fig_2d_y.show()

    # Create 2D slices at specific x values
    for specific_x_value in specific_x_values:
        if specific_x_value in pivot.columns:
            slice_data = pivot[specific_x_value]
            slice_y = slice_data.index
            slice_z = slice_data.values

            # Create a 2D line plot for the slice
            fig_2d_x = go.Figure()
            fig_2d_x.add_trace(go.Scatter(
                x=slice_y,
                y=slice_z,
                mode="lines+markers",
                name=f"Slice at angle_id_7 = {specific_x_value}"
            ))
            fig_2d_x.update_layout(
                title=f"2D Slice at angle_id_7 = {specific_x_value} for RPM Measure: {rpm_1}",
                xaxis_title="Angle Left (angle_id_6)",
                yaxis_title="Avg Absolute Influence",
                template="plotly_white"
            )
            fig_2d_x.show()


In [157]:
# Group by rpm1 (max_rpm_id_0) and angles (angle_id_6, angle_id_7)
averaged_over_rpm0 = df_combined.groupby(["max_rpm_id_0", "angle_id_6", "angle_id_7"])[
    ["absolute_influence", "relative_influence"]
].mean().reset_index()

# Rename columns for clarity
averaged_over_rpm0.rename(columns={
    "absolute_influence": "avg_absolute_influence",
    "relative_influence": "avg_relative_influence"
}, inplace=True)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# Iterate over each unique `max_rpm_id_0` value
unique_rpm0_values = averaged_over_rpm0["max_rpm_id_0"].unique()

for rpm_0 in unique_rpm0_values:
    # Filter the averaged results for the current `max_rpm_id_0` value
    filtered_results = averaged_over_rpm0[averaged_over_rpm0["max_rpm_id_0"] == rpm_0]

    # Create pivot table for plotting
    pivot = filtered_results.pivot(index="angle_id_6", columns="angle_id_7", values="avg_relative_influence")

    # Extract x, y, and z for the surface
    x = pivot.columns.values  # Angle ID 7 (columns of the pivot)
    y = pivot.index.values    # Angle ID 6 (index of the pivot)
    z = pivot.values          # Average relative influence (pivot values)

    # Ensure x, y are 2D for the surface
    x, y = np.meshgrid(x, y)

    # Create a new figure
    fig = go.Figure()

    # Add surface for this motor speed
    fig.add_trace(go.Surface(
        z=z,
        x=x,
        y=y,
        colorscale="Viridis",  # Use a color scale for this surface
        showscale=True,  # Show the color scale
        name=f"RPM 1: {rpm_0}"  # Title for this surface
    ))

    # Update layout
    fig.update_layout(
        title=f"3D Surface Plot for RPM Left: {rpm_0}",
        scene=dict(
            xaxis_title="Angle Measure",
            yaxis_title="Angle Left",
            zaxis_title="Avg Relative Influence"
        ),
        template="plotly_white"
    )

    # Show the plot
    fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go

# Iterate over each unique `max_rpm_id_0` value
unique_rpm0_values = averaged_over_rpm0["max_rpm_id_0"].unique()

for rpm_0 in unique_rpm0_values:
    # Filter the averaged results for the current `max_rpm_id_0` value
    filtered_results = averaged_over_rpm0[averaged_over_rpm0["max_rpm_id_0"] == rpm_0]

    # Create pivot table for plotting
    pivot = filtered_results.pivot(index="angle_id_6", columns="angle_id_7", values="avg_absolute_influence")

    # Extract x, y, and z for the surface
    x = pivot.columns.values  # Angle ID 7 (columns of the pivot)
    y = pivot.index.values    # Angle ID 6 (index of the pivot)
    z = pivot.values          # Average relative influence (pivot values)

    # Ensure x, y are 2D for the surface
    x, y = np.meshgrid(x, y)

    # Create a new figure
    fig = go.Figure()

    # Add surface for this motor speed
    fig.add_trace(go.Surface(
        z=z,
        x=x,
        y=y,
        colorscale="Viridis",  # Use a color scale for this surface
        showscale=True,  # Show the color scale
        name=f"RPM 1: {rpm_0}"  # Title for this surface
    ))

    # Update layout
    fig.update_layout(
        title=f"3D Surface Plot for RPM Left: {rpm_0}",
        scene=dict(
            xaxis_title="Angle Measure",
            yaxis_title="Angle Left",
            zaxis_title="Avg Absolute Influence"
        ),
        template="plotly_white"
    )

    # Show the plot
    fig.show()
